In [1]:
import os
import numpy as np
from mne.io import read_raw_eeglab

def process_set_files(input_folder):
    # Create output directory
    output_folder = "Preprocessed_CSV"
    os.makedirs(output_folder, exist_ok=True)
    
    # Get all .set files
    set_files = [f for f in os.listdir(input_folder) if f.endswith('.set')]
    
    for set_file in set_files:
        print(f"Processing {set_file}...")
        set_path = os.path.join(input_folder, set_file)
        
        try:
            raw = read_raw_eeglab(set_path, preload=True)
        except Exception as e:
            print(f"Error loading {set_file}: {e}")
            continue
        
        # Get EEG data (exclude non-EEG channels)
        eeg_data = raw.get_data(picks='eeg')
        sfreq = raw.info['sfreq']  # Sampling frequency (should be 256Hz)
        n_channels, n_samples = eeg_data.shape
        
        # Calculate time points (0 to 1 second in steps of 1/sfreq)
        time_column = np.arange(0, 1, 1/sfreq).reshape(-1, 1)  # Column vector
        
        # Segment into 1-second chunks
        samples_per_segment = int(sfreq)
        n_segments = n_samples // samples_per_segment
        
        # Get channel names for header
        ch_names = raw.info['ch_names']
        header = "time," + ",".join(ch_names)
        
        # Base filename without extension
        base_name = os.path.splitext(set_file)[0]
        
        for seg_num in range(n_segments):
            start = seg_num * samples_per_segment
            end = start + samples_per_segment
            segment = eeg_data[:, start:end].T  # Transpose to (time, channels)
            
            # Combine time column with EEG data
            segment_with_time = np.hstack((time_column, segment))
            
            # Save as CSV
            csv_name = f"{base_name}_seg_{seg_num + 1}.csv"
            csv_path = os.path.join(output_folder, csv_name)
            
            np.savetxt(
                csv_path,
                segment_with_time,
                delimiter=',',
                header=header,
                comments='',
                fmt='%.6f'  # 6 decimal places for time and EEG values
            )
        
        print(f"Saved {n_segments} segments for {set_file}")
    
    print("All files processed successfully!")

if __name__ == "__main__":
    input_folder = r"C:\Users\umaim\Downloads\preprocessed_data\preprocessed_data"
    if not os.path.exists(input_folder):
        print(f"Error: Folder not found - {input_folder}")
    else:
        process_set_files(input_folder)

Processing 6921143_H S15 EO.set...
Saved 302 segments for 6921143_H S15 EO.set
Processing 6921959_H S15 EO.set...
Saved 302 segments for 6921959_H S15 EO.set
Processing H S1 EC.set...
Saved 300 segments for H S1 EC.set
Processing H S1 EO.set...
Saved 351 segments for H S1 EO.set
Processing H S10 EC.set...
Saved 376 segments for H S10 EC.set
Processing H S10 EO.set...
Saved 300 segments for H S10 EO.set
Processing H S11 EC.set...
Saved 300 segments for H S11 EC.set
Processing H S11 EO.set...
Saved 302 segments for H S11 EO.set
Processing H S12 EO.set...
Saved 300 segments for H S12 EO.set
Processing H S13 EC.set...
Saved 300 segments for H S13 EC.set
Processing H S13 EO.set...
Saved 303 segments for H S13 EO.set
Processing H S14 EC.set...
Saved 302 segments for H S14 EC.set
Processing H S14 EO.set...
Saved 194 segments for H S14 EO.set
Processing H S15 EC.set...
Saved 300 segments for H S15 EC.set
Processing H S16 EC.set...
Saved 292 segments for H S16 EC.set
Processing H S16 EO.set...
